# Analyze voting behaviour

In [1]:
import pandas as pd
from glob import glob
import os

DATA_DIR = "data"
VOTES_DIR = os.path.join(DATA_DIR, "votes")

# Store aggregated analysis data here
EXPORT_DIR = "output"


### Import data

Assumes you have run `1. Collect data.ipynb`.


In [2]:
df_meps = pd.read_csv("data/meps.csv", encoding="utf-8")
df_meps.head()

,Name,European political group,Member State,National Party,Status,link
0,Lars ADAKTUSSON,EPP,Sweden,Kristdemokraterna,Inactive,https://www.votewatch.eu/en/term8-lars-adaktus...
1,Isabella ADINOLFI,EFDD,Italy,Movimento 5 Stelle,Active,https://www.votewatch.eu/en/term8-isabella-adi...
2,Marco AFFRONTE,Greens/EFA,Italy,Independent,Active,https://www.votewatch.eu/en/term8-marco-affron...
3,Laura AGEA,EFDD,Italy,Movimento 5 Stelle,Active,https://www.votewatch.eu/en/term8-laura-agea.html
4,John Stuart AGNEW,ENF,United Kingdom,United Kingdom Independence Party,Active,https://www.votewatch.eu/en/term8-john-stuart-...


In [3]:
df_mep_ids = pd.read_csv("data/mep_ids.csv", encoding="utf-8")
df_mep_ids.head()

,Name,mep_id
0,Lars ADAKTUSSON,1
1,Isabella ADINOLFI,2
2,Marco AFFRONTE,3
3,Laura AGEA,4
4,John Stuart AGNEW,5


In [4]:
df_meps = df_meps.merge(df_mep_ids, on="Name", how="left")
df_meps.head()

,Name,European political group,Member State,National Party,Status,link,mep_id
0,Lars ADAKTUSSON,EPP,Sweden,Kristdemokraterna,Inactive,https://www.votewatch.eu/en/term8-lars-adaktus...,1
1,Isabella ADINOLFI,EFDD,Italy,Movimento 5 Stelle,Active,https://www.votewatch.eu/en/term8-isabella-adi...,2
2,Marco AFFRONTE,Greens/EFA,Italy,Independent,Active,https://www.votewatch.eu/en/term8-marco-affron...,3
3,Laura AGEA,EFDD,Italy,Movimento 5 Stelle,Active,https://www.votewatch.eu/en/term8-laura-agea.html,4
4,John Stuart AGNEW,ENF,United Kingdom,United Kingdom Independence Party,Active,https://www.votewatch.eu/en/term8-john-stuart-...,5


In [5]:
file_paths = glob(os.path.join(VOTES_DIR, "*.csv"))
dfs = []
for file_path in file_paths:
    _df = pd.read_csv(file_path, encoding="utf-8")
    mep_id = file_path.split("/")[-1].replace(".csv", "")
    _df["mep_id"] = int(mep_id)
    dfs.append(_df)
votes_df = pd.concat(dfs, sort=False)
votes_df["has_made_correction"] = ~votes_df["correction"].isna()

votes_df["vote"] = votes_df["vote"].str.replace("This MEP ","")

# Possible votes are: 'voted against', 'voted in favour', 'voted abstained',
# 'did not vote', 'was absent' and 'Documented absence'
# We mark the first four as "present"
votes_df["was_present"] = votes_df["vote"].isin([u'voted against', u'voted in favour', u'voted abstained', 'did not vote'])

# "Participated" = made an active vote
votes_df["participated"] = votes_df["vote"].isin([u'voted against', u'voted in favour', u'voted abstained'])


# Add MEP data
votes_df = votes_df.merge(df_meps, how="left", on="mep_id")
votes_df.head()

,Unnamed: 0,act_id,correction,date,domain,euro_group_loyalty,euro_tara_loyalty,vote,mep_id,has_made_correction,was_present,participated,Name,European political group,Member State,National Party,Status,link
0,0,10072.0,NaN,2019-04-04,"Civil liberties, justice & home affairs",Rebel,Loyal,voted against,409,False,True,True,Linda McAVAN,S&D,United Kingdom,Labour Party,Active,https://www.votewatch.eu/en/term8-linda-mcavan...
1,1,10073.0,NaN,2019-04-04,Employment & social affairs,Loyal,Rebel,voted in favour,409,False,True,True,Linda McAVAN,S&D,United Kingdom,Labour Party,Active,https://www.votewatch.eu/en/term8-linda-mcavan...
2,2,10074.0,NaN,2019-04-04,Transport & tourism,Loyal,Rebel,voted against,409,False,True,True,Linda McAVAN,S&D,United Kingdom,Labour Party,Active,https://www.votewatch.eu/en/term8-linda-mcavan...
3,3,10075.0,NaN,2019-04-04,Transport & tourism,Loyal,Rebel,voted against,409,False,True,True,Linda McAVAN,S&D,United Kingdom,Labour Party,Active,https://www.votewatch.eu/en/term8-linda-mcavan...
4,4,10076.0,NaN,2019-04-04,Transport & tourism,Loyal,Rebel,voted against,409,False,True,True,Linda McAVAN,S&D,United Kingdom,Labour Party,Active,https://www.votewatch.eu/en/term8-linda-mcavan...


### Who has to correct their votes most often?

In [6]:
def aggregate_wrong_votes(group_by):
    wrong_vote_df = votes_df[votes_df["has_made_correction"]].pivot_table(index=group_by, columns="vote", aggfunc="count", values="date")
    wrong_vote_df["n_corrections"] = wrong_vote_df.sum(axis=1)
    # "feltryckningar" = röstat för/emot och ändrat sig
    wrong_vote_df["n_wrong_votes"] = wrong_vote_df[["voted against", "voted in favour", "voted abstained"]].sum(axis=1)
    n_votes = votes_df.groupby(group_by)["Name"].count().rename("n_votes_total")

    n_present_votes = votes_df.groupby(group_by)["was_present"].sum().rename("n_present_votes_total")
    n_absent_votes = n_votes - n_present_votes
    presence = n_present_votes / n_votes

    n_participation_votes = votes_df.groupby(group_by)["participated"].sum().rename("n_participation_votes")
    participation = n_participation_votes / n_votes
 
    n_no_vote = votes_df[votes_df["vote"] == "did not vote"].groupby(group_by)["Name"].count().rename("n_no_vote")
    no_vote_share = n_no_vote / n_present_votes

    wrong_vote_df = pd.concat([
        wrong_vote_df,
        n_present_votes.to_frame(),
        n_absent_votes.rename("n_absent_votes").to_frame(),
        n_votes.to_frame(),
        presence.rename("presence").to_frame(),
        participation.rename("participation").to_frame(),
        n_no_vote,
        no_vote_share.rename("no_vote_share"),
    ], axis=1, sort=False)
    wrong_vote_df["share_corrections"] = wrong_vote_df["n_corrections"] / wrong_vote_df["n_present_votes_total"]
    wrong_vote_df["share_wrong_votes"] = wrong_vote_df["n_wrong_votes"] / wrong_vote_df["n_present_votes_total"]
    
    return wrong_vote_df.sort_values("share_wrong_votes")

# Make a bunch of aggregations
by_country = aggregate_wrong_votes("Member State")
by_mep = aggregate_wrong_votes("Name")
by_mep = by_mep.merge(df_meps.set_index("Name")[["European political group", "Member State", "National Party"]], how="left", left_index=True, right_index=True)
by_ep_group = aggregate_wrong_votes("European political group")
by_domain = aggregate_wrong_votes("domain")
by_nat_party = aggregate_wrong_votes("National Party")

# Add country to party aggregation
party_country_lookup = votes_df.groupby(["National Party", "Member State"]).size().to_frame().reset_index().iloc[:, :2]
by_nat_party = by_nat_party.merge(party_country_lookup, how="left", left_index=True, right_on="National Party").set_index("National Party")


In [11]:
COLUMN_NAMES = {
    "en": {
        "n_votes_total": u"Number of votes in total",
        "n_present_votes_total": u"Number of votings when MEP was present",
        "presence": u"Presence (%)",
        "participation": u"Particiation (%)",
        "n_wrong_votes": u"Number of 'wrong votes'",
        "share_wrong_votes": u"Share of votes that were 'wrong' (of the votings where the MEP was present)",
        "n_corrections": u"Number of corrected votes in total",
        "share_corrections": u"Share of votes that were corrected (of the votings where the MEP was present)",
        
    },
    "sv": {
        "n_votes_total": u"Antal voteringar totalt",
        "n_present_votes_total": u"Antal voteringar med närvaro",
        "presence": u"Närvaro (%)",
        "participation": u"Deltagande (%)",
        "n_wrong_votes": u"Antal feltryck",
        "share_wrong_votes": u"Andel feltryck (av voteringar där MEP varit närvarande)",
        "n_corrections": u"Antal rättningar",
        "share_corrections": u"Andel rättningar (av voteringar där MEP varit närvarande)",

        "did not vote": u"Ursprunglig röst: Röstade inte",
        "voted abstained": u"Ursprunglig röst: Avstod",
        "voted against": u"Ursprunglig röst: Röstade emot",
        "voted in favour": u"Ursprunglig röst: Röstade för",
    }
}

def export_wrong_vote_df(df, file_name, lang="en"):
    file_path = os.path.join(EXPORT_DIR, u"{} - {}.csv".format(file_name, lang))
    print u"Save to {}".format(file_path)
    columns = [u'n_votes_total', "n_present_votes_total", "presence", "participation", u'n_wrong_votes', 
               u'share_wrong_votes', u'n_corrections', u'share_corrections', ]

    if "European political group" in df.columns:
        columns.append("European political group")
    if "Member State" in df.columns:
        columns.append("Member State")

    column_translation = COLUMN_NAMES[lang]
        
    (df[columns]
    .sort_values("share_wrong_votes", ascending=False) 
    .rename(columns=column_translation)
    .to_csv(file_path, encoding="utf-8")
    )

export_wrong_vote_df(by_country, "corrected_votes_by_country")
export_wrong_vote_df(by_mep, "corrected_votes_by_mep")
export_wrong_vote_df(by_ep_group, "corrected_votes_by_ep_group")
export_wrong_vote_df(by_domain, "corrected_votes_by_domain")
export_wrong_vote_df(by_nat_party, "corrected_votes_by_nat_party")


Save to output/corrected_votes_by_country - en.csv
Save to output/corrected_votes_by_mep - en.csv
Save to output/corrected_votes_by_ep_group - en.csv
Save to output/corrected_votes_by_domain - en.csv
Save to output/corrected_votes_by_nat_party - en.csv


In [8]:
by_country

,did not vote,voted abstained,voted against,voted in favour,n_corrections,n_wrong_votes,n_present_votes_total,n_absent_votes,n_votes_total,presence,participation,n_no_vote,no_vote_share,share_corrections,share_wrong_votes
Member State,,,,,,,,,,,,,,,
Estonia,8,4,12,19,43,35,56755.0,3539.0,60294,0.941304,0.869009,4359,0.076804,0.000758,0.000617
Croatia,18,7,37,33,95,77,104682.0,5857.0,110539,0.947014,0.906359,4494,0.042930,0.000908,0.000736
Luxembourg,27,3,23,19,72,45,57449.0,2845.0,60294,0.952815,0.892908,3612,0.062873,0.001253,0.000783
Germany,163,85,404,440,1092,929,904639.0,58761.0,963400,0.939007,0.878587,58208,0.064344,0.001207,0.001027
Latvia,5,10,25,48,88,83,73938.0,6292.0,80230,0.921575,0.849931,5748,0.077741,0.001190,0.001123
Lithuania,11,28,31,60,130,119,98133.0,12344.0,110477,0.888266,0.842184,5091,0.051879,0.001325,0.001213
Poland,227,70,289,262,848,621,483483.0,27918.0,511401,0.945409,0.905464,20428,0.042252,0.001754,0.001284
Romania,52,20,183,242,497,445,288690.0,32720.0,321410,0.898199,0.847105,16422,0.056885,0.001722,0.001541
Bulgaria,10,25,109,110,254,244,156658.0,14124.0,170782,0.917298,0.872662,7623,0.048660,0.001621,0.001558


In [10]:
by_mep[by_mep["Member State"] == "Sweden"].sort_values("no_vote_share", ascending=False)[["n_present_votes_total","n_no_vote","no_vote_share"]]

,n_present_votes_total,n_no_vote,no_vote_share
Isabella LÖVIN,82.0,38.0,0.463415
Marit PAULSEN,1565.0,177.0,0.113099
Soraya POST,9697.0,914.0,0.094256
Christofer FJELLNER,8675.0,772.0,0.088991
Gunnar HÖKMARK,9710.0,840.0,0.086509
Cecilia WIKSTRÖM,9022.0,752.0,0.083352
Anders SELLSTRÖM This person became MEP later than the general start of the term.,2227.0,168.0,0.075438
Anna Maria CORAZZA BILDT,8540.0,610.0,0.071429
Linnéa ENGSTRÖM This person became MEP later than the general start of the term.,7329.0,492.0,0.067131
Anna HEDH,8689.0,571.0,0.065715
